In [1]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import math
import os
import torch
import random
import pandas as pd
import numpy as np
from sklearn import datasets, ensemble
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate, train_test_split
import itertools
import math
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,V_WIND,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,30.0,28,73100,...,3.77,15.9,2.730798,12,42.01,43.16,40.96,1407.668330,0.001660,3.048333
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,30.0,15,37900,...,-6.72,24.5,4.289058,10,67.53,64.55,59.34,2089.046774,0.001614,17.138611
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,50.0,7,115000,...,0.00,9.4,0.000000,14,65.30,66.39,56.94,603.193047,0.001743,98.827500
3,TRAIN_000003,JP,TMR7,Cargo,0.000000,2020-09-18 22:06,B726632,10.0,33,1490,...,-7.31,22.1,4.693735,7,43.02,43.15,41.11,1169.853455,0.000069,0.000000
4,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,30.0,10,27600,...,2.31,22.8,2.345875,14,90.45,93.65,88.11,1107.944894,0.000197,96.030556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,40.0,13,93200,...,NaN,NaN,NaN,6,61.25,62.21,55.70,1333.609109,0.000360,65.850000
367437,TRAIN_367437,JP,QYY1,Tanker,0.000000,2022-04-29 2:58,D847216,10.0,9,1280,...,0.87,17.1,1.028558,11,105.37,109.34,104.69,1955.103846,0.000552,0.000000
367438,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,30.0,6,25000,...,3.36,31.7,2.557156,15,97.73,99.10,95.78,1601.291086,0.002615,0.997500
367439,TRAIN_367439,JP,TMR7,Cargo,0.000000,2020-12-22 10:07,N211282,10.0,8,2400,...,-2.44,10.8,3.055715,19,49.75,50.08,47.02,1191.353331,0.000069,0.000000


In [3]:
# 1. Load the USD data
usd_data = pd.read_csv("USD.csv")

# 2. Convert the '날짜' column in USD data to datetime format
#usd_data['날짜'] = pd.to_datetime(usd_data['날짜'], format='%Y- %m- %d').dt.date
usd_data['날짜'] = pd.to_datetime(usd_data['날짜'], format='%Y-%m-%d').dt.date

# Convert the 'ATA' column in train and test data to only the date format
train['ATA_date'] = pd.to_datetime(train['ATA']).dt.date
test['ATA_date'] = pd.to_datetime(test['ATA']).dt.date

# 3. Merge the USD data with train and test data on the date columns
train_merged = pd.merge(train, usd_data, left_on='ATA_date', right_on='날짜', how='left')
test_merged = pd.merge(test, usd_data, left_on='ATA_date', right_on='날짜', how='left')

# Drop the '날짜' and 'ATA_date' columns from merged datasets as they are redundant
train_merged.drop(['날짜', 'ATA_date'], axis=1, inplace=True)
test_merged.drop(['날짜', 'ATA_date'], axis=1, inplace=True)
train_merged.drop(['거래량', '변동 %'],axis=1,inplace=True)
test_merged.drop(['거래량', '변동 %'],axis=1,inplace=True)
train_merged.drop(['시가', '고가', '저가'],axis=1,inplace=True)
test_merged.drop(['시가', '고가', '저가'],axis=1,inplace=True)
train_merged['종가'] = train_merged['종가'].str.replace(',', '').astype(float)
test_merged['종가'] = test_merged['종가'].str.replace(',', '').astype(float)
train = train_merged.copy()
test = test_merged.copy()
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR,종가
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,30.0,28,73100,...,15.9,2.730798,12,42.01,43.16,40.96,1407.668330,0.001660,3.048333,1145.14
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,30.0,15,37900,...,24.5,4.289058,10,67.53,64.55,59.34,2089.046774,0.001614,17.138611,1186.62
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,50.0,7,115000,...,9.4,0.000000,14,65.30,66.39,56.94,603.193047,0.001743,98.827500,1122.99
3,TRAIN_000003,JP,TMR7,Cargo,0.000000,2020-09-18 22:06,B726632,10.0,33,1490,...,22.1,4.693735,7,43.02,43.15,41.11,1169.853455,0.000069,0.000000,1164.81
4,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,30.0,10,27600,...,22.8,2.345875,14,90.45,93.65,88.11,1107.944894,0.000197,96.030556,1301.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367436,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,40.0,13,93200,...,NaN,NaN,6,61.25,62.21,55.70,1333.609109,0.000360,65.850000,1120.96
367437,TRAIN_367437,JP,QYY1,Tanker,0.000000,2022-04-29 2:58,D847216,10.0,9,1280,...,17.1,1.028558,11,105.37,109.34,104.69,1955.103846,0.000552,0.000000,1263.03
367438,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,30.0,6,25000,...,31.7,2.557156,15,97.73,99.10,95.78,1601.291086,0.002615,0.997500,1316.41
367439,TRAIN_367439,JP,TMR7,Cargo,0.000000,2020-12-22 10:07,N211282,10.0,8,2400,...,10.8,3.055715,19,49.75,50.08,47.02,1191.353331,0.000069,0.000000,1109.72


# 0포함 o x

In [34]:
train = train[train['DIST'] != 0]
train = train.reset_index(drop = True)
train

,SAMPLE_ID,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ATA,ID,BREADTH,BUILT,DEADWEIGHT,...,AIR_TEMPERATURE,BN,ATA_LT,DUBAI,BRENT,WTI,BDI_ADJ,PORT_SIZE,CI_HOUR,종가
0,TRAIN_000000,CN,EKP8,Bulk,30.736578,2020-10-15 4:03,Z517571,30.0,28,73100,...,15.9,2.730798,12,42.01,43.16,40.96,1407.668330,0.001660,3.048333,1145.14
1,TRAIN_000001,CN,EUC8,Container,63.220425,2019-09-17 2:55,U467618,30.0,15,37900,...,24.5,4.289058,10,67.53,64.55,59.34,2089.046774,0.001614,17.138611,1186.62
2,TRAIN_000002,CN,NGG6,Container,90.427421,2019-02-23 6:43,V378315,50.0,7,115000,...,9.4,0.000000,14,65.30,66.39,56.94,603.193047,0.001743,98.827500,1122.99
3,TRAIN_000004,RU,NNC2,Container,8.813725,2022-08-13 12:57,D215135,30.0,10,27600,...,22.8,2.345875,14,90.45,93.65,88.11,1107.944894,0.000197,96.030556,1301.31
4,TRAIN_000005,CN,NGG6,Container,81.435335,2015-09-08 14:24,Z156413,30.0,22,18100,...,NaN,NaN,22,45.75,48.89,45.92,820.288044,0.001743,42.078056,1194.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220050,TRAIN_367433,IN,UJM2,Bulk,30.199074,2022-03-23 8:35,Y242521,30.0,2,63500,...,36.5,4.306719,14,111.93,120.65,113.90,2077.159292,0.000217,53.400833,1218.62
220051,TRAIN_367434,CN,QQW1,Bulk,55.408765,2022-06-16 14:27,D236761,30.0,16,26500,...,28.2,2.651752,22,108.43,114.13,109.56,2067.433444,0.000595,83.960833,1288.23
220052,TRAIN_367436,CN,YRT6,Bulk,59.018184,2017-11-11 22:23,J661243,40.0,13,93200,...,NaN,NaN,6,61.25,62.21,55.70,1333.609109,0.000360,65.850000,1120.96
220053,TRAIN_367438,SG,GIW5,Container,1.768630,2022-07-14 7:58,Q635545,30.0,6,25000,...,31.7,2.557156,15,97.73,99.10,95.78,1601.291086,0.002615,0.997500,1316.41


In [4]:
train.drop(['SAMPLE_ID'],axis=1,inplace=True)
test.drop(['SAMPLE_ID'],axis=1,inplace=True)

In [5]:
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])
train['year'] = train['ATA'].dt.year
train['month'] = train['ATA'].dt.month
train['day'] = train['ATA'].dt.day
train['weekday'] = train['ATA'].dt.dayofweek
train['rounded_hour'] = (train['ATA'].dt.hour + (train['ATA'].dt.minute // 30)).apply(lambda x: 0 if x == 24 else x)
test['year'] = test['ATA'].dt.year
test['month'] = test['ATA'].dt.month
test['day'] = test['ATA'].dt.day
test['weekday'] = test['ATA'].dt.dayofweek
test['rounded_hour'] = (test['ATA'].dt.hour + (test['ATA'].dt.minute // 30)).apply(lambda x: 0 if x == 24 else x)

# sin, cos 변환 함수 정의
def encode_cyclic_feature(data, column, max_val):
    data[column + '_sin'] = np.sin(2 * np.pi * data[column] / max_val)
    data[column + '_cos'] = np.cos(2 * np.pi * data[column] / max_val)
    return data

# 각 피처에 대해 sin, cos 변환 수행
train = encode_cyclic_feature(train, 'month', 12)
train = encode_cyclic_feature(train, 'day', 31)
train = encode_cyclic_feature(train, 'weekday', 7)
train = encode_cyclic_feature(train, 'rounded_hour', 24)
test = encode_cyclic_feature(test, 'month', 12)
test = encode_cyclic_feature(test, 'day', 31)
test = encode_cyclic_feature(test, 'weekday', 7)
test = encode_cyclic_feature(test, 'rounded_hour', 24)

train.drop(['ATA'],axis=1,inplace=True)
test.drop(['ATA'],axis=1,inplace=True)

test[['year', 'month', 'day', 'weekday', 'rounded_hour']].head()

,year,month,day,weekday,rounded_hour
0,2022,8,27,5,8
1,2022,3,27,6,21
2,2023,1,18,2,2
3,2016,8,2,1,1
4,2023,1,24,1,0


In [6]:
train.drop(['DEADWEIGHT', 'DEPTH','ARI_CO','ARI_PO','SHIP_TYPE_CATEGORY','DIST','ID','BREADTH','BUILT','DRAUGHT','GT','LENGTH','SHIPMANAGER','FLAG','U_WIND','V_WIND','AIR_TEMPERATURE','BN','ATA_LT','year','month','day','weekday','rounded_hour','PORT_SIZE'],axis=1,inplace=True)
test.drop(['DEADWEIGHT', 'DEPTH', 'ARI_CO','ARI_PO','SHIP_TYPE_CATEGORY','DIST','ID','BREADTH','BUILT','DRAUGHT','GT','LENGTH','SHIPMANAGER','FLAG','U_WIND','V_WIND','AIR_TEMPERATURE','BN','ATA_LT','year','month','day','weekday','rounded_hour','PORT_SIZE'],axis=1,inplace=True)

In [7]:
train.drop(['rounded_hour_sin','rounded_hour_cos','weekday_sin','weekday_cos'],axis=1,inplace=True)
test.drop(['rounded_hour_sin','rounded_hour_cos','weekday_sin','weekday_cos'],axis=1,inplace=True)

### 여기 위까지 11   /  0같이 학습 시 10

In [8]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 1. Select the features for PCA
features = ['WTI', 'BRENT', 'DUBAI']
train_subset = train[features]
test_subset = test[features]

# 3. Apply PCA and get 2 principal components
pca = PCA(n_components=1)
train_pca = pca.fit_transform(train_subset)
test_pca = pca.transform(test_subset)

# 4. Add the principal components to the train and test data
train['PCA1'] = train_pca[:, 0]
test['PCA1'] = test_pca[:, 0]

In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 1. Select the features for PCA
features = ['WTI', '종가']
train_subset = train[features]
test_subset = test[features]

# 3. Apply PCA and get 2 principal components
pca = PCA(n_components=1)
train_pca = pca.fit_transform(train_subset)
test_pca = pca.transform(test_subset)

# 4. Add the principal components to the train and test data
train['PCA2'] = train_pca[:, 0]
test['PCA2'] = test_pca[:, 0]

In [10]:
# 1. Polynomial features
train['DUBAI_squared'] = train['DUBAI'] ** 2
train['BRENT_squared'] = train['BRENT'] ** 2
train['WTI_squared'] = train['WTI'] ** 2

# 2. Ratios
train['DUBAI_WTI_ratio'] = train['DUBAI'] / train['WTI']
train['BRENT_WTI_ratio'] = train['BRENT'] / train['WTI']

# 1. Polynomial features
test['DUBAI_squared'] = test['DUBAI'] ** 2
test['BRENT_squared'] = test['BRENT'] ** 2
test['WTI_squared'] = test['WTI'] ** 2

# 2. Ratios
test['DUBAI_WTI_ratio'] = test['DUBAI'] / test['WTI']
test['BRENT_WTI_ratio'] = test['BRENT'] / test['WTI']

train['oil_price_mean'] = train[['DUBAI', 'BRENT', 'WTI']].mean(axis=1)
test['oil_price_mean'] = test[['DUBAI', 'BRENT', 'WTI']].mean(axis=1)


In [11]:
# 2. Ratios with oil prices
train['BDI_ADJ_DUBAI_ratio'] = train['BDI_ADJ'] / train['DUBAI']
train['BDI_ADJ_BRENT_ratio'] = train['BDI_ADJ'] / train['BRENT']
train['BDI_ADJ_WTI_ratio'] = train['BDI_ADJ'] / train['WTI']

test['BDI_ADJ_DUBAI_ratio'] = test['BDI_ADJ'] / test['DUBAI']
test['BDI_ADJ_BRENT_ratio'] = test['BDI_ADJ'] / test['BRENT']
test['BDI_ADJ_WTI_ratio'] = test['BDI_ADJ'] / test['WTI']

# Display the first few rows of the transformed training dataset after adding new interaction features
train.head()


,DUBAI,BRENT,WTI,BDI_ADJ,CI_HOUR,종가,month_sin,month_cos,day_sin,day_cos,...,PCA2,DUBAI_squared,BRENT_squared,WTI_squared,DUBAI_WTI_ratio,BRENT_WTI_ratio,oil_price_mean,BDI_ADJ_DUBAI_ratio,BDI_ADJ_BRENT_ratio,BDI_ADJ_WTI_ratio
0,42.01,43.16,40.96,1407.668330,3.048333,1145.14,-0.866025,5.000000e-01,0.101168,-0.994869,...,-29.171610,1764.8401,1862.7856,1677.7216,1.025635,1.053711,42.043333,33.507935,32.615114,34.366903
1,67.53,64.55,59.34,2089.046774,17.138611,1186.62,-1.000000,-1.836970e-16,-0.299363,-0.954139,...,14.511907,4560.3009,4166.7025,3521.2356,1.138018,1.087799,63.806667,30.935092,32.363234,35.204698
2,65.30,66.39,56.94,603.193047,98.827500,1122.99,0.866025,5.000000e-01,-0.998717,-0.050649,...,-48.806535,4264.0900,4407.6321,3242.1636,1.146821,1.165964,62.876667,9.237260,9.085601,10.593485
3,43.02,43.15,41.11,1169.853455,0.000000,1164.81,-1.000000,-1.836970e-16,-0.485302,-0.874347,...,-9.682637,1850.7204,1861.9225,1690.0321,1.046461,1.049623,42.426667,27.193246,27.111320,28.456664
4,90.45,93.65,88.11,1107.944894,96.030556,1301.31,-0.866025,-5.000000e-01,0.485302,-0.874347,...,132.138770,8181.2025,8770.3225,7763.3721,1.026558,1.062876,90.736667,12.249253,11.830698,12.574565


# 여까지

In [55]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(train)
test_data = TabularDataset(test)


#predictor = TabularPredictor(label='CI_HOUR',  eval_metric='mean_absolute_error').fit(train, presets='medium_quality',  ag_args_fit={'num_gpus': 0}, included_model_types = ['RF','GBM','XGB'])
predictor = TabularPredictor(label='CI_HOUR',  eval_metric='mean_absolute_error').fit(train, presets='medium_quality',  ag_args_fit={'num_gpus': 0})

predictor.leaderboard(train, silent=True)


y_pred = predictor.predict(test_data)
y_pred = pd.DataFrame(y_pred, columns=['CI_HOUR'])
submission['CI_HOUR'] = y_pred['CI_HOUR']
predictor.leaderboard(train, silent=True)

No path specified. Models will be saved in: "AutogluonModels\ag-20231013_010835\"
Presets specified: ['medium_quality']
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231013_010835\"
AutoGluon Version:  0.8.2
Python Version:     3.8.6
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Disk Space Avail:   75.63 GB / 498.62 GB (15.2%)
Train Data Rows:    220055
Train Data Columns: 11
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2159.130556, 0.002777778, 103.30825, 210.47157)
	If 'regression' is not the correct problem_type, please manually s

[1000]	valid_set's l1: 63.0481
[2000]	valid_set's l1: 51.3807
[3000]	valid_set's l1: 44.6376
[4000]	valid_set's l1: 40.1058
[5000]	valid_set's l1: 36.8774
[6000]	valid_set's l1: 34.589
[7000]	valid_set's l1: 32.659
[8000]	valid_set's l1: 31.1117
[9000]	valid_set's l1: 29.7672
[10000]	valid_set's l1: 28.5996


	-28.5975	 = Validation score   (-mean_absolute_error)
	23.47s	 = Training   runtime
	0.29s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l1: 47.483
[2000]	valid_set's l1: 33.1863
[3000]	valid_set's l1: 26.1733
[4000]	valid_set's l1: 22.4433
[5000]	valid_set's l1: 20.0248
[6000]	valid_set's l1: 18.3516
[7000]	valid_set's l1: 17.1177
[8000]	valid_set's l1: 16.3392
[9000]	valid_set's l1: 15.6508
[10000]	valid_set's l1: 15.1717


	-15.1715	 = Validation score   (-mean_absolute_error)
	17.71s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-13.4291	 = Validation score   (-mean_absolute_error)
	41.88s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: CatBoost ...
	-25.3636	 = Validation score   (-mean_absolute_error)
	229.98s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-13.1244	 = Validation score   (-mean_absolute_error)
	11.36s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-75.0309	 = Validation score   (-mean_absolute_error)
	89.78s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost ...
	-78.4901	 = Validation score   (-mean_absolute_error)
	0.29s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-37.6122	 = Validation score   (-mean_absolute_error)
	981.57s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting mod

[1000]	valid_set's l1: 28.9797
[2000]	valid_set's l1: 18.0876
[3000]	valid_set's l1: 14.5018
[4000]	valid_set's l1: 12.9483
[5000]	valid_set's l1: 12.1523
[6000]	valid_set's l1: 11.668
[7000]	valid_set's l1: 11.4397
[8000]	valid_set's l1: 11.2641
[9000]	valid_set's l1: 11.1501
[10000]	valid_set's l1: 11.0836


	-11.0826	 = Validation score   (-mean_absolute_error)
	30.82s	 = Training   runtime
	0.32s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-11.014	 = Validation score   (-mean_absolute_error)
	0.15s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1433.3s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20231013_010835\")


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist,-6.591493,-23.284874,0.444403,0.013012,0.815742,0.444403,0.013012,0.815742,1,True,2
1,LightGBMLarge,-7.245406,-11.082593,30.634839,0.322293,30.816002,30.634839,0.322293,30.816002,1,True,11
2,WeightedEnsemble_L2,-7.447660,-11.014002,32.170233,0.360328,42.318454,0.011010,0.000000,0.146132,2,True,12
3,RandomForestMSE,-10.134063,-13.429098,1.647497,0.039035,41.877053,1.647497,0.039035,41.877053,1,True,5
4,ExtraTreesMSE,-10.553143,-13.124426,1.524385,0.038035,11.356319,1.524385,0.038035,11.356319,1,True,7
5,LightGBM,-13.061882,-15.171476,17.376790,0.202184,17.708091,17.376790,0.202184,17.708091,1,True,4
6,CatBoost,-25.245778,-25.363623,0.235214,0.005004,229.979964,0.235214,0.005004,229.979964,1,True,6
7,LightGBMXT,-29.551170,-28.597470,24.349126,0.286260,23.473330,24.349126,0.286260,23.473330,1,True,3
8,KNeighborsUnif,-29.909579,-31.610439,0.454413,0.015013,0.747680,0.454413,0.015013,0.747680,1,True,1
9,NeuralNetTorch,-40.861173,-37.612200,0.574522,0.010009,981.571129,0.574522,0.010009,981.571129,1,True,10


In [43]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(train)
test_data = TabularDataset(test)


#predictor = TabularPredictor(label='CI_HOUR',  eval_metric='mean_absolute_error').fit(train, presets='medium_quality',  ag_args_fit={'num_gpus': 0}, included_model_types = ['RF','GBM','XGB'])
predictor = TabularPredictor(label='CI_HOUR',  eval_metric='mean_absolute_error').fit(train, presets='best_quality',  ag_args_fit={'num_gpus': 0})

y_pred_best = predictor.predict(test_data)
y_pred_best = pd.DataFrame(y_pred_best, columns=['CI_HOUR'])
submission['CI_HOUR'] = y_pred_best['CI_HOUR']
predictor.leaderboard(train, silent=True)

No path specified. Models will be saved in: "AutogluonModels\ag-20231014_125112\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231014_125112\"
AutoGluon Version:  0.8.2
Python Version:     3.8.6
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Disk Space Avail:   92.29 GB / 498.62 GB (18.5%)
Train Data Rows:    220055
Train Data Columns: 20
Label Column: CI_HOUR
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2159.130556, 0.002777778, 

KeyboardInterrupt: 

In [44]:
submission[submission['CI_HOUR'] > 500]

,SAMPLE_ID,CI_HOUR
65,TEST_000065,922.186829
67,TEST_000067,667.122498
76,TEST_000076,617.021118
82,TEST_000082,590.948669
188,TEST_000188,1215.226562
...,...,...
244838,TEST_244838,1361.329224
244897,TEST_244897,550.927917
244910,TEST_244910,1247.631348
244945,TEST_244945,885.531555


In [ ]:
# 1. Get indices of samples in submission where CI_HOUR is greater than 500
indices_to_refit = submission[submission['CI_HOUR'] >= 500].index

# 2. Train a model on the entire training data using MSE as the evaluation metric
predictor_mse = TabularPredictor(label='CI_HOUR', eval_metric='root_mean_squared_error').fit(train, presets='best_quality', ag_args_fit={'num_gpus': 0})

# 3. Predict only for the selected samples using the model trained with MSE
test_samples_to_refit = test_data.iloc[indices_to_refit]
y_pred_refit = predictor_mse.predict(test_samples_to_refit)

# 4. Replace the values in the original submission with the new predictions
submission.loc[indices_to_refit, 'CI_HOUR'] = y_pred_refit.values
submission

In [ ]:
predictor.leaderboard(train, silent=True)

In [14]:
predictor.leaderboard(train, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,-4.705637,-18.651591,1.369853,1.322202,2.369352,1.369853,1.322202,2.369352,1,True,2
1,RandomForestMSE_BAG_L2,-6.087152,-7.843315,10.826452,31.640868,368.139901,2.218017,8.556647,161.189589,2,True,6
2,WeightedEnsemble_L3,-6.087152,-7.843315,10.831456,31.645873,369.781198,0.005005,0.005005,1.641297,3,True,8
3,ExtraTreesMSE_BAG_L2,-6.194661,-8.111462,10.418909,30.463945,252.829004,1.810474,7.379725,45.878691,2,True,7
4,RandomForestMSE_BAG_L1,-7.428190,-10.608984,2.966654,10.619943,152.603943,2.966654,10.619943,152.603943,1,True,3
5,WeightedEnsemble_L2,-7.441861,-10.475369,5.740220,20.401750,204.205781,0.007007,0.004004,2.019999,2,True,5
6,ExtraTreesMSE_BAG_L1,-7.640721,-10.693210,2.766559,9.777804,49.581840,2.766559,9.777804,49.581840,1,True,4
7,KNeighborsUnif_BAG_L1,-19.081965,-24.261792,1.505368,1.364272,2.395178,1.505368,1.364272,2.395178,1,True,1


In [ ]:
predictor.feature_importance(train_data)

Computing feature importance via permutation shuffling for 15 features using 5000 rows with 5 shuffle sets...


	121.11s	= Expected runtime (24.22s per shuffle set)
	23.42s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
BDI_ADJ,700.538737,4.610450,2.251132e-10,5,710.031718,691.045755
month_sin,611.403242,9.600074,7.291651e-09,5,631.169927,591.636557
month_cos,606.572601,3.499666,1.329649e-10,5,613.778461,599.366741
종가,594.202767,4.975863,5.900319e-10,5,604.448137,583.957397
WTI,434.767073,4.513935,1.394163e-09,5,444.061328,425.472818
BRENT_WTI_ratio,423.840708,7.220452,1.010323e-08,5,438.707720,408.973697
WTI_squared,393.544277,3.415519,6.807537e-10,5,400.576878,386.511676
DUBAI_squared,386.989083,4.064721,1.460311e-09,5,395.358399,378.619767
DUBAI,374.958851,2.891071,4.240795e-10,5,380.911606,369.006096
BRENT_squared,324.761469,4.213908,3.400682e-09,5,333.437965,316.084974


In [43]:
test = pd.read_csv('test.csv')
all = pd.concat([test,submission],axis=1)
#all['CI_HOUR'][all['DIST'] == 0] = 0
all.loc[all['DIST'] == 0, 'CI_HOUR'] = 0
submission['CI_HOUR'] = all['CI_HOUR']
submission['CI_HOUR'][submission['CI_HOUR'] < 0] = 0
submission.to_csv('best_include_0.csv',index=False)
submission

,SAMPLE_ID,CI_HOUR
0,TEST_000000,93.535583
1,TEST_000001,389.344513
2,TEST_000002,0.000000
3,TEST_000003,0.000000
4,TEST_000004,2.585824
...,...,...
244984,TEST_244984,46.205944
244985,TEST_244985,489.423889
244986,TEST_244986,0.000000
244987,TEST_244987,0.000000


# CAT

In [124]:
from xgboost import XGBRegressor
#import xgboost as xgb
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import numpy as np


def objective(trial: Trial, X_train, y_train, X_val, y_val):
    params = {
            'iterations':trial.suggest_int("iterations", 300, 1000),
            'learning_rate' : trial.suggest_uniform('learning_rate',0.1, 1),
            'depth': trial.suggest_int('depth',5, 16),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
            'reg_lambda': trial.suggest_uniform('reg_lambda',30,100),
            'subsample': trial.suggest_uniform('subsample',0.3,1),
            'random_strength': trial.suggest_uniform('random_strength',10,100),
            'od_wait':trial.suggest_int('od_wait', 10, 150),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,20),
            'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 1, 100),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0., 1.0),
            'random_state' : 724,
            'verbose' : 0,
        }
    #'task_type' : 'GPU',
    #"eval_metric":'RMSE',
    cat = CatBoostRegressor(**params)
    cat.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_val,y_val)],cat_features=cat_features,
              verbose=False)
    cat_pred = cat.predict(X_val)
    score = mean_absolute_error(y_val, cat_pred)
    return score


In [40]:
from sklearn.preprocessing import LabelEncoder
import bisect
'''
# Categorical 컬럼 인코딩
categorical_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
    le = LabelEncoder()
    train[feature] = le.fit_transform(train[feature].astype(str))
    le_classes_set = set(le.classes_)
    test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
    le_classes = le.classes_.tolist()
    bisect.insort_left(le_classes, '-1')
    le.classes_ = np.array(le_classes)
    test[feature] = le.transform(test[feature].astype(str))
    encoders[feature] = le
cat_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG']

# 결측치 처리
train_x.fillna(train_x.mean(), inplace=True)
test.fillna(train_x.mean(), inplace=True)
'''
cat_features = ['ARI_CO', 'ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER', 'FLAG', 'country_cluster', 'PORT_SIZE_Zone', 'BREADTH', 'DEPTH', 'DRAUGHT']

# 수치형 변수만 대상으로 결측치 대체
numeric_cols = train_x.select_dtypes(include=[np.number]).columns

# 훈련 데이터의 평균 계산
mean_values = train_x[numeric_cols].mean()

# 결측치 대체
train_x[numeric_cols] = train_x[numeric_cols].fillna(mean_values)
test[numeric_cols] = test[numeric_cols].fillna(mean_values)

In [126]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220055 entries, 0 to 220054
Data columns (total 57 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ARI_CO                 220055 non-null  object 
 1   ARI_PO                 220055 non-null  object 
 2   SHIP_TYPE_CATEGORY     220055 non-null  object 
 3   DIST                   220055 non-null  float64
 4   ID                     220055 non-null  object 
 5   BREADTH                220055 non-null  object 
 6   BUILT                  220055 non-null  int64  
 7   DEADWEIGHT             220055 non-null  int64  
 8   DEPTH                  220055 non-null  object 
 9   DRAUGHT                220055 non-null  object 
 10  GT                     220055 non-null  int64  
 11  LENGTH                 220055 non-null  float64
 12  SHIPMANAGER            220055 non-null  object 
 13  FLAG                   220055 non-null  object 
 14  U_WIND                 220055 non-nu

In [127]:
# 모든 카테고리형 변수를 문자열로 변환
categorical_cols = train_x.select_dtypes(include=['object', 'category']).columns
train_x[categorical_cols] = train_x[categorical_cols].astype(str)
test[categorical_cols] = test[categorical_cols].astype(str)


In [128]:
kf = KFold(n_splits=5, shuffle=True, random_state=724)
fold_results = []
test_predictions = []

# 각 Fold에 대해서 Optuna로 하이퍼파라미터 튜닝
for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    X_train, X_val = train_x.iloc[train_index, :], train_x.iloc[val_index, :]
    y_train, y_val = train_y.iloc[train_index, :], train_y.iloc[val_index, :]

    print(f"Optimizing hyperparameters for fold {fold+1}...")
    study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=724))
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val.values), n_trials=20)

    best_params = study.best_params
    best_score = study.best_value
    
    print(f"Best MAE for fold {fold+1}: {best_score}")
    print(f"Best hyperparameters for fold {fold+1}: {best_params}")
    
    fold_results.append({
        'fold': fold+1,
        'best_score': best_score,
        'best_params': best_params
    })
    
    # 최적의 하이퍼파라미터로 테스트 데이터 예측
    model = CatBoostRegressor(**best_params)
    model.fit(X_train, y_train, cat_features=cat_features)
    test_pred = model.predict(test)
    test_predictions.append(test_pred)

# 평균 앙상블
final_prediction = np.mean(test_predictions, axis=0)

# 결과 출력
for result in fold_results:
    print(f"Fold {result['fold']}, Best MAE: {result['best_score']}, Best hyperparameters: {result['best_params']}")

# 최종 예측 저장
final_prediction_df = pd.DataFrame(final_prediction, columns=['CI_HOUR'])
submission['CI_HOUR'] = final_prediction

[I 2023-10-05 23:13:14,949] A new study created in memory with name: no-name-d5bdd458-3bc8-4f77-9659-7cde3598a38d


Optimizing hyperparameters for fold 1...


[I 2023-10-05 23:15:17,832] Trial 0 finished with value: 55.43213467773245 and parameters: {'iterations': 921, 'learning_rate': 0.7313877507251003, 'depth': 10, 'min_data_in_leaf': 15, 'reg_lambda': 30.77074089604671, 'subsample': 0.5286843392696118, 'random_strength': 54.372999344451735, 'od_wait': 51, 'leaf_estimation_iterations': 18, 'bagging_temperature': 1.0913808956698392, 'colsample_bylevel': 0.36753066317430727}. Best is trial 0 with value: 55.43213467773245.
[I 2023-10-05 23:17:27,695] Trial 1 finished with value: 46.73698461577896 and parameters: {'iterations': 433, 'learning_rate': 0.5899605678803301, 'depth': 9, 'min_data_in_leaf': 15, 'reg_lambda': 62.223289044475145, 'subsample': 0.9759050658077331, 'random_strength': 30.55733714009755, 'od_wait': 125, 'leaf_estimation_iterations': 20, 'bagging_temperature': 10.099229114607697, 'colsample_bylevel': 0.8290967984097537}. Best is trial 1 with value: 46.73698461577896.
[I 2023-10-05 23:19:36,021] Trial 2 finished with value: 

Best RMSE for fold 1: 42.25324087037929
Best hyperparameters for fold 1: {'iterations': 792, 'learning_rate': 0.4094644098911695, 'depth': 8, 'min_data_in_leaf': 13, 'reg_lambda': 43.85829584203627, 'subsample': 0.6745303236399682, 'random_strength': 85.6116646925902, 'od_wait': 62, 'leaf_estimation_iterations': 16, 'bagging_temperature': 51.306767359876446, 'colsample_bylevel': 0.4699924477380927}
0:	learn: 210.0952407	total: 96.6ms	remaining: 1m 16s
1:	learn: 210.0940552	total: 147ms	remaining: 58.1s
2:	learn: 210.0938004	total: 208ms	remaining: 54.6s
3:	learn: 209.1358980	total: 362ms	remaining: 1m 11s
4:	learn: 207.6132118	total: 492ms	remaining: 1m 17s
5:	learn: 206.2839777	total: 653ms	remaining: 1m 25s
6:	learn: 206.0279903	total: 791ms	remaining: 1m 28s
7:	learn: 204.0681873	total: 966ms	remaining: 1m 34s
8:	learn: 204.0638360	total: 1.01s	remaining: 1m 28s
9:	learn: 203.4280451	total: 1.22s	remaining: 1m 35s
10:	learn: 203.1901894	total: 1.39s	remaining: 1m 38s
11:	learn: 203.

[I 2023-10-06 00:47:49,969] A new study created in memory with name: no-name-34d199d9-2296-4ac7-aab4-4affaab2fe81


...

Best RMSE for fold 4: 42.99479569369386
Best hyperparameters for fold 4: {'iterations': 792, 'learning_rate': 0.4094644098911695, 'depth': 8, 'min_data_in_leaf': 13, 'reg_lambda': 43.85829584203627, 'subsample': 0.6745303236399682, 'random_strength': 85.6116646925902, 'od_wait': 62, 'leaf_estimation_iterations': 16, 'bagging_temperature': 51.306767359876446, 'colsample_bylevel': 0.4699924477380927}
0:	learn: 207.5462183	total: 148ms	remaining: 1m 56s
1:	learn: 206.2793843	total: 235ms	remaining: 1m 32s
2:	learn: 206.2774985	total: 287ms	remaining: 1m 15s
3:	learn: 204.7584634	total: 371ms	remaining: 1m 13s
4:	learn: 204.1595926	total: 541ms	remaining: 1m 25s
5:	learn: 204.1418121	total: 604ms	remaining: 1m 19s
6:	learn: 204.1093265	total: 700ms	remaining: 1m 18s
7:	learn: 202.6380865	total: 845ms	remaining: 1m 22s
8:	learn: 202.1132612	total: 1.02s	remaining: 1m 28s
9:	learn: 201.9492653	total: 1.1s	remaining: 1m 26s
10:	learn: 201.9431037	total: 1.16s	remaining: 1m 22s
11:	learn: 201.

[I 2023-10-06 05:35:54,142] A new study created in memory with name: no-name-9a45990b-9b73-4ff6-9786-80073f391832


Optimizing hyperparameters for fold 5...


[I 2023-10-06 05:38:13,591] Trial 0 finished with value: 57.20639981749408 and parameters: {'iterations': 921, 'learning_rate': 0.7313877507251003, 'depth': 10, 'min_data_in_leaf': 15, 'reg_lambda': 30.77074089604671, 'subsample': 0.5286843392696118, 'random_strength': 54.372999344451735, 'od_wait': 51, 'leaf_estimation_iterations': 18, 'bagging_temperature': 1.0913808956698392, 'colsample_bylevel': 0.36753066317430727}. Best is trial 0 with value: 57.20639981749408.
[I 2023-10-06 05:40:26,699] Trial 1 finished with value: 46.48582358805089 and parameters: {'iterations': 433, 'learning_rate': 0.5899605678803301, 'depth': 9, 'min_data_in_leaf': 15, 'reg_lambda': 62.223289044475145, 'subsample': 0.9759050658077331, 'random_strength': 30.55733714009755, 'od_wait': 125, 'leaf_estimation_iterations': 20, 'bagging_temperature': 10.099229114607697, 'colsample_bylevel': 0.8290967984097537}. Best is trial 1 with value: 46.48582358805089.
[I 2023-10-06 05:42:35,209] Trial 2 finished with value: 

Best RMSE for fold 5: 43.48887650381214
Best hyperparameters for fold 5: {'iterations': 783, 'learning_rate': 0.2584880573013857, 'depth': 12, 'min_data_in_leaf': 30, 'reg_lambda': 45.02231140060744, 'subsample': 0.6753500090427036, 'random_strength': 21.788829786258407, 'od_wait': 90, 'leaf_estimation_iterations': 16, 'bagging_temperature': 58.67357912276298, 'colsample_bylevel': 0.5379457040607726}
0:	learn: 205.2582218	total: 697ms	remaining: 9m 5s
1:	learn: 204.2043019	total: 886ms	remaining: 5m 45s
2:	learn: 203.0624090	total: 1.01s	remaining: 4m 23s
3:	learn: 202.0806996	total: 1.23s	remaining: 3m 59s
4:	learn: 200.1313413	total: 1.92s	remaining: 4m 59s
5:	learn: 198.2753633	total: 2.61s	remaining: 5m 37s
6:	learn: 197.5619923	total: 3.39s	remaining: 6m 16s
7:	learn: 197.5466068	total: 3.45s	remaining: 5m 34s
8:	learn: 197.1022517	total: 3.64s	remaining: 5m 13s
9:	learn: 196.9068203	total: 3.75s	remaining: 4m 50s
10:	learn: 196.8625524	total: 3.9s	remaining: 4m 33s
11:	learn: 195

In [129]:
all = pd.concat([test,submission],axis=1)
all['CI_HOUR'][all['DIST'] == 0] = 0
submission['CI_HOUR'] = all['CI_HOUR']
submission['CI_HOUR'][submission['CI_HOUR'] < 0] = 0
submission.to_csv('CAT_v2_724.csv',index=False)
submission

,SAMPLE_ID,CI_HOUR
0,TEST_000000,85.724445
1,TEST_000001,290.797561
2,TEST_000002,0.000000
3,TEST_000003,0.000000
4,TEST_000004,57.593767
...,...,...
244984,TEST_244984,69.481320
244985,TEST_244985,418.025254
244986,TEST_244986,0.000000
244987,TEST_244987,0.000000


In [58]:
train

,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,nAcid,nBase,SpAbs_A,...,MACCS_key_158,MACCS_key_159,MACCS_key_160,MACCS_key_161,MACCS_key_162,MACCS_key_163,MACCS_key_164,MACCS_key_165,MLM,HLM
0,3.259,400.495,5,2,8,3.259,117.37,0,0,35.689316,...,1,1,1,1,1,1,1,1,26.010,50.680
1,2.169,301.407,2,1,2,2.172,73.47,0,0,26.575899,...,1,0,1,1,1,1,1,1,29.270,50.590
2,1.593,297.358,5,0,3,1.585,62.45,2,1,29.802128,...,1,0,1,1,1,1,0,1,5.586,80.892
3,4.771,494.652,6,0,5,3.475,92.60,0,1,45.884166,...,1,1,1,1,1,1,1,1,5.710,2.000
4,2.335,268.310,3,0,1,2.337,42.43,0,0,26.308663,...,1,1,1,1,1,1,1,1,93.270,99.990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3466,3.409,396.195,3,1,5,3.409,64.74,0,0,30.902711,...,1,0,1,1,1,0,1,1,1.556,3.079
3467,1.912,359.381,4,1,3,1.844,77.37,0,0,35.887372,...,1,1,1,1,1,1,1,1,35.560,47.630
3468,1.941,261.320,3,1,6,2.124,70.14,0,0,23.546531,...,1,1,1,1,1,1,1,1,56.150,1.790
3469,0.989,284.696,5,1,5,0.989,91.51,0,0,23.936088,...,1,1,0,1,1,1,1,1,0.030,2.770


# TabNet

In [50]:
import optuna
import numpy as np
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# Objective 함수 정의
def objective(trial, X_train, y_train, X_val, y_val):
    n_d = trial.suggest_int("n_d", 4, 64)
    n_a = trial.suggest_int("n_a", 4, 64)
    n_steps = trial.suggest_int("n_steps", 1, 10)
    gamma = trial.suggest_float("gamma", 1.0, 2.0)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-5, 1e-3, log=True)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    optimizer_params = {"lr": lr}
    
    model = TabNetRegressor(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        gamma=gamma,
        lambda_sparse=lambda_sparse,
        optimizer_params=optimizer_params
    )
    
    y_train_2d = y_train.values.reshape(-1, 1)
    y_val_2d = y_val.values.reshape(-1, 1)
    
    model.fit(
        X_train=X_train.values, 
        y_train=y_train_2d,
        eval_set=[(X_val.values, y_val_2d)],
        eval_metric=['mae'], 
        max_epochs=50, 
        patience=10
    )
    
    preds_val = model.predict(X_val.values)
    mae = mean_absolute_error(y_val.values.ravel(), preds_val)
    return mae

In [45]:
# 모든 카테고리형 변수를 문자열로 변환
categorical_cols = train_x.select_dtypes(include=['object', 'category']).columns
train_x[categorical_cols] = train_x[categorical_cols].astype(str)
test[categorical_cols] = test[categorical_cols].astype(str)
# object 타입 피처만 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
test_unknown = test.copy()

for col in train_x.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    train_x[col] = le.fit_transform(train_x[col])
    
    known_mask = test[col].isin(le.classes_)
    test_unknown = test_unknown[~known_mask]  # 새로운 레이블을 포함하는 데이터 포인트를 별도로 저장
    test = test[known_mask]
    
    test[col] = le.transform(test[col])
    label_encoders[col] = le

In [47]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [51]:
kf = KFold(n_splits=5, shuffle=True, random_state=724)
fold_results = []
test_predictions = []

for fold, (train_index, val_index) in enumerate(kf.split(train_x)):
    X_train, X_val = train_x.iloc[train_index, :], train_x.iloc[val_index, :]
    y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]

    print(f"Optimizing hyperparameters for fold {fold+1}...")
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=724))
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=20)

    best_params = study.best_params
    best_score = study.best_value
    
    print(f"Best MAE for fold {fold+1}: {best_score}")
    print(f"Best hyperparameters for fold {fold+1}: {best_params}")
    
    fold_results.append({
        'fold': fold+1,
        'best_score': best_score,
        'best_params': best_params
    })
    
    # 최적의 하이퍼파라미터로 테스트 데이터 예측
    model = TabNetRegressor(**best_params)
    model.fit(X_train.values, y_train.values.ravel(), max_epochs=100)
    test_pred = model.predict(test.values)
    test_predictions.append(test_pred)

final_prediction = np.mean(test_predictions, axis=0)

for result in fold_results:
    print(f"Fold {result['fold']}, Best MAE: {result['best_score']}, Best hyperparameters: {result['best_params']}")

[I 2023-10-08 01:57:59,512] A new study created in memory with name: no-name-a4896c90-e6f1-42a6-8c8b-5140d4f7cb5f


Optimizing hyperparameters for fold 1...
epoch 0  | loss: 54667.76901| val_0_mae: 180.92095|  0:00:12s
epoch 1  | loss: 54281.59527| val_0_mae: 96.3394 |  0:00:23s
epoch 2  | loss: 53814.79642| val_0_mae: 95.458  |  0:00:35s
epoch 3  | loss: 53561.90072| val_0_mae: 94.85518|  0:00:47s
epoch 4  | loss: 53108.98061| val_0_mae: 93.88412|  0:00:59s
epoch 5  | loss: 52801.55621| val_0_mae: 93.0834 |  0:01:10s
epoch 6  | loss: 52390.89816| val_0_mae: 92.26418|  0:01:21s
epoch 7  | loss: 52056.07431| val_0_mae: 91.56768|  0:01:32s
epoch 8  | loss: 51702.23693| val_0_mae: 91.00958|  0:01:43s
epoch 9  | loss: 51476.37852| val_0_mae: 90.44323|  0:01:54s
epoch 10 | loss: 51061.01298| val_0_mae: 89.9735 |  0:02:06s
epoch 11 | loss: 50774.96551| val_0_mae: 89.50916|  0:02:18s
epoch 12 | loss: 50536.75444| val_0_mae: 89.09913|  0:02:29s
epoch 13 | loss: 50189.4527| val_0_mae: 88.73736|  0:02:41s
epoch 14 | loss: 49885.91626| val_0_mae: 88.31281|  0:02:52s
epoch 15 | loss: 49535.19113| val_0_mae: 88.

[I 2023-10-08 02:04:09,494] Trial 0 finished with value: 87.4515860027796 and parameters: {'n_d': 58, 'n_a': 46, 'n_steps': 5, 'gamma': 1.471768994831443, 'lambda_sparse': 1.0520131499846483e-05, 'lr': 9.551591485457036e-05}. Best is trial 0 with value: 87.4515860027796.


epoch 0  | loss: 55200.8212| val_0_mae: 140.91934|  0:00:18s
epoch 1  | loss: 54819.6298| val_0_mae: 104.29145|  0:00:35s
epoch 2  | loss: 54864.47222| val_0_mae: 99.48548|  0:00:54s
epoch 3  | loss: 54627.99508| val_0_mae: 98.81957|  0:01:13s
epoch 4  | loss: 54556.0473| val_0_mae: 98.27959|  0:01:32s
epoch 5  | loss: 54470.91333| val_0_mae: 97.81953|  0:01:51s
epoch 6  | loss: 54251.61036| val_0_mae: 97.32818|  0:02:10s
epoch 7  | loss: 54020.38877| val_0_mae: 96.90192|  0:02:29s
epoch 8  | loss: 53893.73273| val_0_mae: 96.4771 |  0:02:48s
epoch 9  | loss: 53832.84439| val_0_mae: 96.04991|  0:03:07s
epoch 10 | loss: 53692.07525| val_0_mae: 95.65555|  0:03:27s
epoch 11 | loss: 53605.63461| val_0_mae: 95.25395|  0:03:47s
epoch 12 | loss: 53461.50434| val_0_mae: 94.9103 |  0:04:06s
epoch 13 | loss: 53462.24061| val_0_mae: 94.5333 |  0:04:24s
epoch 14 | loss: 53168.84983| val_0_mae: 94.22555|  0:04:46s
epoch 15 | loss: 53158.89853| val_0_mae: 93.87284|  0:05:05s
epoch 16 | loss: 52942.24

[I 2023-10-08 02:21:06,939] Trial 1 finished with value: 87.50547043557528 and parameters: {'n_d': 34, 'n_a': 21, 'n_steps': 9, 'gamma': 1.0189881736525601, 'lambda_sparse': 5.433270488189108e-05, 'lr': 3.736575233292654e-05}. Best is trial 0 with value: 87.4515860027796.


...

[I 2023-10-08 02:38:26,739] Trial 3 finished with value: 87.22914795206387 and parameters: {'n_d': 53, 'n_a': 62, 'n_steps': 6, 'gamma': 1.8290967984097537, 'lambda_sparse': 2.0638315057558745e-05, 'lr': 0.00014753325905081117}. Best is trial 3 with value: 87.22914795206387.


epoch 0  | loss: 55057.72515| val_0_mae: 103.21543|  0:00:19s
epoch 1  | loss: 54999.30688| val_0_mae: 97.88925|  0:00:39s
epoch 2  | loss: 54923.15769| val_0_mae: 99.3274 |  0:00:59s
epoch 3  | loss: 54882.61184| val_0_mae: 99.66273|  0:01:18s
epoch 4  | loss: 54781.98376| val_0_mae: 99.4892 |  0:01:38s
epoch 5  | loss: 54834.16983| val_0_mae: 99.36418|  0:01:57s
epoch 6  | loss: 54784.86199| val_0_mae: 99.13258|  0:02:17s
epoch 7  | loss: 54676.25836| val_0_mae: 98.90912|  0:02:36s
epoch 8  | loss: 54621.33027| val_0_mae: 98.75119|  0:02:56s
epoch 9  | loss: 54569.33982| val_0_mae: 98.64019|  0:03:16s
epoch 10 | loss: 54499.69737| val_0_mae: 98.42384|  0:03:36s
epoch 11 | loss: 54499.32717| val_0_mae: 98.35121|  0:03:56s

Early stopping occurred at epoch 11 with best_epoch = 1 and best_val_0_mae = 97.88925


[I 2023-10-08 02:42:55,642] Trial 4 finished with value: 97.88925116629748 and parameters: {'n_d': 33, 'n_a': 8, 'n_steps': 9, 'gamma': 1.3954106897914667, 'lambda_sparse': 3.435899500927241e-05, 'lr': 1.4530706847817623e-05}. Best is trial 3 with value: 87.22914795206387.


epoch 0  | loss: 46095.39365| val_0_mae: 417.88197|  0:00:18s
epoch 1  | loss: 43015.46217| val_0_mae: 136.88145|  0:00:36s
epoch 2  | loss: 42736.49585| val_0_mae: 103.93545|  0:00:55s
epoch 3  | loss: 42568.07205| val_0_mae: 100.54177|  0:01:13s
epoch 4  | loss: 42609.04317| val_0_mae: 98.73855|  0:01:30s
epoch 5  | loss: 42485.57384| val_0_mae: 106.69002|  0:01:48s
epoch 6  | loss: 41993.5897| val_0_mae: 101.21619|  0:02:07s


KeyboardInterrupt: 